In [234]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import patsy

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split, cross_val_score

In [235]:
df = pd.read_csv('/Users/supermaxfederman/DSI-NYC-2/curriculum/week-05/1.3-classification-challenge/classification_challenge_training.csv') # read in the census-derived data; mind the separator
df.head()

,Unnamed: 0,age,type_employer,education,education_num,marital,occupation,relationship,race,sex,capital_gain,capital_loss,hr_per_week,country,income
0,4,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
1,14656,32,Private,HS-grad,9,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,40,United-States,1
2,26677,29,Private,10th,6,Married-spouse-absent,Adm-clerical,Unmarried,White,Female,0,0,40,Mexico,0
3,4281,30,Private,Assoc-voc,11,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,Mexico,0
4,15668,66,Private,HS-grad,9,Widowed,Priv-house-serv,Not-in-family,White,Female,0,0,8,United-States,0


In [259]:
dft= pd.read_csv('/Users/supermaxfederman/DSI-NYC-2/curriculum/week-05/1.3-classification-challenge/classification_challenge_submission.csv')
dft.head()

,Unnamed: 0,age,type_employer,education,education_num,marital,occupation,relationship,race,sex,capital_gain,capital_loss,hr_per_week,country
0,18099,45,Federal-gov,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,Black,Male,0,0,20,United-States
1,10672,17,Private,12th,8,Never-married,Sales,Own-child,White,Male,0,0,40,United-States
2,17947,24,Private,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Male,0,0,40,United-States
3,13753,46,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,Black,Male,4064,0,40,United-States
4,4279,59,Private,Bachelors,13,Separated,Adm-clerical,Unmarried,White,Male,0,0,60,United-States


In [236]:
# Check values and their counts
pd.set_option('display.max_rows', 500)
df.apply(lambda c: c.value_counts()).T.stack()

/Applications/anaconda/lib/python3.5/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: int() < str(), sort order is undefined for incomparable objects
  result = result.union(other)
/Applications/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:5274: RuntimeWarning: unorderable types: int() < str(), sort order is undefined for incomparable objects
  index = _union_indexes(indexes)
/Applications/anaconda/lib/python3.5/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: str() > int(), sort order is undefined for incomparable objects
  result = result.union(other)


Unnamed: 0    1                                 1.0
              2                                 1.0
              4                                 1.0
              5                                 1.0
              6                                 1.0
              7                                 1.0
              8                                 1.0
              9                                 1.0
              10                                1.0
              12                                1.0
              14                                1.0
              15                                1.0
              17                                1.0
              18                                1.0
              19                                1.0
              20                                1.0
              21                                1.0
              22                                1.0
              23                                1.0
            

In [237]:
df.relationship.unique()

array(['Husband', 'Wife', 'Unmarried', 'Not-in-family', 'Own-child',
       'Other-relative'], dtype=object)

In [238]:
df

,Unnamed: 0,age,type_employer,education,education_num,marital,occupation,relationship,race,sex,capital_gain,capital_loss,hr_per_week,country,income
0,4,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
1,14656,32,Private,HS-grad,9,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,40,United-States,1
2,26677,29,Private,10th,6,Married-spouse-absent,Adm-clerical,Unmarried,White,Female,0,0,40,Mexico,0
3,4281,30,Private,Assoc-voc,11,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,Mexico,0
4,15668,66,Private,HS-grad,9,Widowed,Priv-house-serv,Not-in-family,White,Female,0,0,8,United-States,0
5,6215,84,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,20,United-States,0
6,25434,24,Private,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Male,2174,0,45,United-States,0
7,21217,42,State-gov,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,48,United-States,1
8,9913,31,Private,Some-college,10,Divorced,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
9,15713,25,Private,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,0,0,40,United-States,1


In [239]:
"""Clean/munge your data."""
X = patsy.dmatrix(' C(capital_gain) + C(education) + C(age) + C(marital)', df)
y = df['income'].values

In [240]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=77)

In [241]:
lr = LogisticRegression(solver='liblinear')
lr_model = lr.fit(X_train, y_train)

In [242]:
lr_ypred = lr_model.predict(X_test)

In [243]:
lr_cm = confusion_matrix(y_test, lr_ypred, labels=lr.classes_)
lr_cm = pd.DataFrame(lr_cm, columns=lr.classes_, index=lr.classes_)
lr_cm

,0,1
0,5774,355
1,842,1088


In [244]:
print(classification_report(y_test, lr_ypred, labels=lr.classes_))

             precision    recall  f1-score   support

          0       0.87      0.94      0.91      6129
          1       0.75      0.56      0.65      1930

avg / total       0.84      0.85      0.84      8059



In [245]:
cvs1 = cross_val_score(lr, X, y, cv=3, scoring='f1_weighted')
cvs1
cvs1.mean()

0.84232971375135879

In [246]:
""" Fit a binary classification predictor."""

y_pred = ?.predict(X_test)
y_score = ?.decision_function(X_test) # Submit these response, the output of model.decision_function

SyntaxError: invalid syntax (<ipython-input-246-57fec3848b73>, line 3)

In [247]:
""" Check your performance so far."""

conmat = np.array(confusion_matrix(Y_test, y_pred, labels=[1,0]))
confusion = pd.DataFrame(conmat, index=['over_50k', 'under_50k'],
                            columns=['predicted_over50k','predicted_under50k'])

print(confusion)
print classification_report(Y_test,y_pred)
roc_auc_score(Y_test, y_score)

""" Plot AUC"""

FPR = dict()
TPR = dict()
ROC_AUC = dict()

# For class 1, find the area under the curve
FPR[1], TPR[1], _ = roc_curve(Y_test, y_score)
ROC_AUC[1] = auc(FPR[1], TPR[1])

# Plot of a ROC curve for class 1 (has_cancer)
plt.figure(figsize=[11,9])
plt.plot(FPR[1], TPR[1], label='ROC curve (area = %0.2f)' % ROC_AUC[1], linewidth=4)
plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Receiver operating characteristic for high/low income', fontsize=18)
plt.legend(loc="lower right")
plt.show()

SyntaxError: invalid syntax (<ipython-input-247-1c60390d98cb>, line 8)

In [255]:
df['marital'].unique()

array(['Married-civ-spouse', 'Married-spouse-absent', 'Widowed',
       'Never-married', 'Divorced', 'Separated', 'Married-AF-spouse'], dtype=object)

In [257]:
df['occupation'].unique()

array(['Handlers-cleaners', 'Adm-clerical', 'Priv-house-serv',
       'Farming-fishing', 'Exec-managerial', 'Craft-repair',
       'Prof-specialty', 'Other-service', 'Sales', '?', 'Tech-support',
       'Protective-serv', 'Machine-op-inspct', 'Transport-moving',
       'Armed-Forces'], dtype=object)

In [260]:
dft['marital'].unique()

array(['Married-civ-spouse', 'Never-married', 'Separated', 'Divorced',
       'Married-spouse-absent', 'Widowed', 'Married-AF-spouse'], dtype=object)

In [261]:
dft['occupation'].unique()

array(['Adm-clerical', 'Sales', 'Craft-repair', 'Machine-op-inspct',
       'Other-service', 'Prof-specialty', 'Farming-fishing',
       'Transport-moving', 'Exec-managerial', 'Tech-support', '?',
       'Protective-serv', 'Handlers-cleaners', 'Priv-house-serv',
       'Armed-Forces'], dtype=object)

In [269]:
df['age'].unique()

array([53, 32, 29, 30, 66, 84, 24, 42, 31, 25, 41, 36, 39, 33, 37, 17, 26,
       43, 21, 20, 57, 34, 67, 27, 35, 19, 40, 59, 47, 49, 46, 44, 22, 64,
       68, 28, 71, 76, 50, 56, 58, 52, 18, 77, 61, 55, 51, 79, 72, 62, 23,
       38, 48, 45, 70, 65, 63, 54, 60, 75, 74, 83, 73, 69, 80, 90, 78, 81,
       82, 85, 88, 86, 87])